# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [79]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

#output file path for data resource files and figures
outputFilePath = os.path.join("Output-Data")

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [17]:
#read the weather_cities.csv into a dataframe
weather_cities_df = pd.read_csv(os.path.join("Output-Data","weather_cities.csv"),delimiter=",")
weather_cities_df

,Latitude,Longitude,City,Country,Max Temp,Humidity,Wind Speed,Cloudiness,Date
0,-6.22,-57.75,jacareacanga,br,97.79,34.0,3.67,47.0,1.601222e+09
1,-10.45,161.92,kirakira,sb,79.39,82.0,12.64,100.0,1.601222e+09
2,-54.80,-68.30,ushuaia,ar,41.00,65.0,16.11,75.0,1.601222e+09
3,64.25,-15.21,hofn,is,48.20,93.0,13.87,90.0,1.601222e+09
4,-20.33,-40.29,vila velha,br,89.60,46.0,13.87,0.0,1.601221e+09
...,...,...,...,...,...,...,...,...,...
558,1.44,31.34,hoima,ug,69.57,91.0,4.12,38.0,1.601221e+09
559,1.74,98.78,sibolga,id,77.34,72.0,1.54,100.0,1.601221e+09
560,-12.49,130.98,palmerston,au,78.80,83.0,4.70,44.0,1.601221e+09
561,26.56,54.88,bandar-e lengeh,ir,91.40,66.0,9.17,0.0,1.601221e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [18]:
#store locations by latitude and longitude
locations = weather_cities_df[["Latitude","Longitude"]].astype(float)

#drop na values (though these should have been handled in WeatherPy before saving to csv)
#do not rid of the index though - indexes will correspond between locations and humididty
weather_cities_df = weather_cities_df.dropna()

#convert humidity to float
humidity = weather_cities_df["Humidity"].astype(float)
maxHumidity = humidity.max()

In [19]:
#create a humidity heatmap layer
fig = gmaps.figure()

heatLayer_humidity = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity = 100, point_radius = 1)

fig.add_layer(heatLayer_humidity)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [22]:
#set criteria for vacation weather
vacationWeather_df = weather_cities_df.loc[(weather_cities_df["Max Temp"]>70)&
                                           (weather_cities_df["Max Temp"]<80)&
                                           (weather_cities_df["Cloudiness"]==0)&
                                           (weather_cities_df["Wind Speed"]<10)].reset_index(drop=True)
vacationWeather_df

,Latitude,Longitude,City,Country,Max Temp,Humidity,Wind Speed,Cloudiness,Date
0,25.23,77.60,kolaras,in,77.68,70.0,4.09,0.0,1.601222e+09
1,39.05,-27.98,lagoa,pt,71.19,86.0,7.38,0.0,1.601221e+09
2,-20.73,139.50,mount isa,au,73.40,18.0,9.17,0.0,1.601221e+09
3,-25.17,46.08,ambovombe,mg,72.28,70.0,9.13,0.0,1.601221e+09
4,12.45,125.29,anito,ph,79.47,86.0,4.83,0.0,1.601221e+09
5,39.60,-9.07,nazare,pt,71.01,100.0,8.99,0.0,1.601221e+09
6,27.98,-114.06,guerrero negro,mx,73.22,69.0,8.72,0.0,1.601221e+09
7,44.03,-64.72,liverpool,ca,75.99,71.0,1.99,0.0,1.601221e+09
8,-20.32,118.57,port hedland,au,71.60,78.0,9.17,0.0,1.601221e+09
9,-24.09,25.03,letlhakeng,bw,79.03,13.0,9.86,0.0,1.601221e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [71]:
#create dataframe for the hotel information
hotel_df = vacationWeather_df.loc[:,["Latitude","Longitude","City","Country"]]
# hotel_df = pd.DataFrame(hotel_df)
hotel_df

,Latitude,Longitude,City,Country
0,25.23,77.60,kolaras,in
1,39.05,-27.98,lagoa,pt
2,-20.73,139.50,mount isa,au
3,-25.17,46.08,ambovombe,mg
4,12.45,125.29,anito,ph
5,39.60,-9.07,nazare,pt
6,27.98,-114.06,guerrero negro,mx
7,44.03,-64.72,liverpool,ca
8,-20.32,118.57,port hedland,au
9,-24.09,25.03,letlhakeng,bw


In [72]:
#set up lists for hotel information
hotelName = []
hotelAddress = []
hotelRating = []
hotelLat = []
hotelLng = []

for index, row in hotel_df.iterrows():
    #current info
    currentCity = row["City"]
    currentCountry = row["Country"]
    
    #geocoordinates
    target_coordinates = f"{row['Latitude']},{row['Longitude']}"
    target_radius = 5000 #this is in meters
    target_type = "hotel"
    target_keyword = "hotel"

    #set parameters
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "keyword": target_keyword,
        "key": g_key
    }
    
    #base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    #run request using params dictionary
    response = requests.get(base_url,params=params).json()
    #simplify finding results
    results = response['results']
     
    try:
        hotelName.append(results[0]['name'])
        hotelAddress.append(results[0]['vicinity'])
        hotelRating.append(results[0]['rating'])
        hotelLat.append(results[0]['geometry']['location']['lat'])
        hotelLng.append(results[0]['geometry']['location']['lng'])
        print(f"Processeing Record for a {target_keyword} in {currentCity}, {currentCountry}")
    except:
        hotelName.append(np.nan)
        hotelAddress.append(np.nan)
        hotelRating.append(np.nan)
        hotelLat.append(np.nan)
        hotelLng.append(np.nan)
        print(f"No results/{target_keyword} found in {currentCity}, {currentCountry}. Skipping...")    

No results/hotel found in kolaras, in. Skipping...
Processeing Record for a hotel in lagoa, pt
Processeing Record for a hotel in mount isa, au
Processeing Record for a hotel in ambovombe, mg
Processeing Record for a hotel in anito, ph
Processeing Record for a hotel in nazare, pt
Processeing Record for a hotel in guerrero negro, mx
Processeing Record for a hotel in liverpool, ca
Processeing Record for a hotel in port hedland, au
Processeing Record for a hotel in letlhakeng, bw


In [73]:
#add new information into the cities_df
hotel_df["Hotel Name"] = hotelName
hotel_df["Hotel Address"] = hotelAddress
hotel_df["Hotel Rating"] = hotelRating
hotel_df["Hotel Latitude"] = hotelLat
hotel_df["Hotel Longitude"] = hotelLng

hotel_df

,Latitude,Longitude,City,Country,Hotel Name,Hotel Address,Hotel Rating,Hotel Latitude,Hotel Longitude
0,25.23,77.60,kolaras,in,NaN,NaN,NaN,NaN,NaN
1,39.05,-27.98,lagoa,pt,Graciosa Resort - Biosphere Island Hotel,"Porto da Barra, Santa Cruz da Graciosa",4.4,39.085412,-27.997393
2,-20.73,139.50,mount isa,au,ibis Styles Mt Isa Verona,"Cnr Rodeo Dr And, Camooweal St, Mount Isa",3.8,-20.726140,139.492803
3,-25.17,46.08,ambovombe,mg,Hotel Le Source,Ambovombe,1.9,-25.178711,46.087739
4,12.45,125.29,anito,ph,The Yellow House,"Pob, Mapanas",5.0,12.472650,125.256297
5,39.60,-9.07,nazare,pt,Hotel Miramar Sul,"Cam. Real, Nazaré",4.2,39.585085,-9.068151
6,27.98,-114.06,guerrero negro,mx,Hotel Los Caracoles,"Calzada, De La República s/n, Estado 30, Guerr...",4.1,27.967758,-114.028336
7,44.03,-64.72,liverpool,ca,Lane's Privateer Inn & Restaurant,"27 Bristol Ave, Liverpool",4.3,44.041652,-64.718112
8,-20.32,118.57,port hedland,au,The Esplanade Hotel,"2 Anderson St, Port Hedland",4.0,-20.313925,118.576081
9,-24.09,25.03,letlhakeng,bw,Tokey's Lodge,Letlhakeng,2.3,-24.104650,25.027120


In [75]:
#remove NaN rows
#did not drop indexes for locations without hotel found - can see how many hotels are found for top locations
hotelFound_df = hotel_df.dropna()
hotelFound_df

,Latitude,Longitude,City,Country,Hotel Name,Hotel Address,Hotel Rating,Hotel Latitude,Hotel Longitude
1,39.05,-27.98,lagoa,pt,Graciosa Resort - Biosphere Island Hotel,"Porto da Barra, Santa Cruz da Graciosa",4.4,39.085412,-27.997393
2,-20.73,139.50,mount isa,au,ibis Styles Mt Isa Verona,"Cnr Rodeo Dr And, Camooweal St, Mount Isa",3.8,-20.726140,139.492803
3,-25.17,46.08,ambovombe,mg,Hotel Le Source,Ambovombe,1.9,-25.178711,46.087739
4,12.45,125.29,anito,ph,The Yellow House,"Pob, Mapanas",5.0,12.472650,125.256297
5,39.60,-9.07,nazare,pt,Hotel Miramar Sul,"Cam. Real, Nazaré",4.2,39.585085,-9.068151
6,27.98,-114.06,guerrero negro,mx,Hotel Los Caracoles,"Calzada, De La República s/n, Estado 30, Guerr...",4.1,27.967758,-114.028336
7,44.03,-64.72,liverpool,ca,Lane's Privateer Inn & Restaurant,"27 Bristol Ave, Liverpool",4.3,44.041652,-64.718112
8,-20.32,118.57,port hedland,au,The Esplanade Hotel,"2 Anderson St, Port Hedland",4.0,-20.313925,118.576081
9,-24.09,25.03,letlhakeng,bw,Tokey's Lodge,Letlhakeng,2.3,-24.104650,25.027120


In [76]:
print(f"There were {len(hotel_df)} locations matching vacation requirements and {len(hotelFound_df)} hotels found.")

There were 10 locations matching vacation requirements and 9 hotels found.


In [77]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotelFound_info = [info_box_template.format(**row) for index, row in hotelFound_df.iterrows()]
locations = hotelFound_df[["Hotel Latitude", "Hotel Longitude"]]

In [80]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotelFound_info)
fig.add_layer(markers)

#save figure
plt.savefig(os.path.join(outputFilePath,"Hotels_Vacation.png"))

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>